In [1]:
import sys
import os

# moce two directories up and make that current directory
os.chdir("../..")
# print current directory
print(os.getcwd())

/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager


In [2]:
from transformers import BeitImageProcessor, BeitForMaskedImageModeling
import torch
from PIL import Image

/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
image = Image.open('tests/LayoutAnalysis/testimage.png').convert('RGB')

In [4]:
processor = BeitImageProcessor.from_pretrained("microsoft/dit-large")
model = BeitForMaskedImageModeling.from_pretrained("microsoft/dit-large")

num_patches = (model.config.image_size // model.config.patch_size) ** 2
pixel_values = processor(images=image, return_tensors="pt").pixel_values
# create random boolean mask of shape (batch_size, num_patches)
bool_masked_pos = torch.randint(low=0, high=2, size=(1, num_patches)).bool()

/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/venv/lib/python3.11/site-packages/transformers/models/beit/image_processing_beit.py:110: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
/Users/chinmayshrivastava/Documents/GitHub/RepoStateManager/venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
outputs = model(pixel_values, bool_masked_pos=bool_masked_pos)
loss, logits = outputs.loss, outputs.logits

In [6]:
outputs.__dict__.keys()

dict_keys(['loss', 'logits', 'hidden_states', 'attentions'])

In [6]:
outputs

MaskedLMOutput(loss=None, logits=tensor([[[-2.2292, -0.8189, -1.8319,  ..., -2.3554, -7.1658, -4.4531],
         [-3.6786,  2.8921,  0.1778,  ..., -2.7646, -4.8025, -2.0717],
         [-2.8232, -1.0928, -1.0393,  ..., -4.7206, -4.2741, -3.4600],
         ...,
         [-2.8232, -1.0928, -1.0393,  ..., -4.7206, -4.2741, -3.4600],
         [-2.8232, -1.0928, -1.0393,  ..., -4.7206, -4.2741, -3.4600],
         [-5.2593, -3.2461, -3.2294,  ..., -4.7179, -7.4076, -6.9522]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [7]:
logits.shape

torch.Size([1, 196, 8192])

In [9]:
# visualize the image with the identified mask
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

# reshape logits and mask
logits = logits.detach().cpu().numpy()
bool_masked_pos = bool_masked_pos[0].detach().cpu().numpy()

# get the index of the most probable masked patch
masked_patch_index = np.argmax(logits[0, bool_masked_pos])
# get the index of the least probable masked patch
unmasked_patch_index = np.argmin(logits[0, ~bool_masked_pos])

# get the coordinates of the patches
patch_coord = processor.get_patch_coordinates(image_size=image.size[0], patch_size=model.config.patch_size)
masked_patch_coord = patch_coord[bool_masked_pos]
unmasked_patch_coord = patch_coord[~bool_masked_pos]

# plot the image
fig, ax = plt.subplots(1)
ax.imshow(image)
# plot the masked patch in red
rect = patches.Rectangle(
    masked_patch_coord[masked_patch_index], model.config.patch_size, model.config.patch_size, linewidth=1, edgecolor="r", facecolor="none"
)
ax.add_patch(rect)
# plot the unmasked patch in green
rect = patches.Rectangle(
    unmasked_patch_coord[unmasked_patch_index], model.config.patch_size, model.config.patch_size, linewidth=1, edgecolor="g", facecolor="none"
)
ax.add_patch(rect)
plt.show()

AttributeError: 'BeitImageProcessor' object has no attribute 'get_patch_coordinates'